In [1]:
import geopandas as gpd
import pandas as pd
import pandas
import math
from shapely.geometry import Point, LineString, Polygon, MultiPoint, MultiLineString, MultiPolygon
from geographiclib.geodesic import Geodesic
from shapely.ops import split
from shapely import ops
import shapely
from pyproj import Proj, Transformer

In [2]:
buildings= gpd.read_file('../tests/data/buildings.geojson')
road= gpd.read_file('../tests/data/road.geojson')

In [3]:
# for d in buildings.columns:
#     print(d)
road['end'] = None
road['start'] = None
road['start_point_x'] = None
road['start_point_y'] = None
road['end_point_x'] = None
road['end_point_y'] = None
road['length'] = None
road['bearing'] = None
road['intersected'] = None
road['mother_road']='None'

buildings['road_direction'] = 'None'
buildings['from_mother_road_direction'] = 'None'

buildings['distance2_intersecting_road'] = 0
buildings['road_length_d_code'] = ''

# print(buildings)
# print(road)

In [4]:
class bdcPoint:
    	
	def __init__(self):
		
		self.x = 0
		self.y = 0

# Constant integers for directions
RIGHT = 1
LEFT = -1
ZERO = 0

def directionOfPoint(A, B, P):
    """[Provides direction of the point in respect to line]

    Args:
        A ([type]): [Starting Point of Line]
        B ([type]): [Ending Point of Line]
        P ([type]): [Point of Interest]

    Returns:
        [Direction(int)]: [Left,Right and Zero : -1,1,0 ( Left direction, Right Direction , On Line)]
    """
    global RIGHT, LEFT, ZERO
	
	# Subtracting co-ordinates of
	# point A from B and P, to
	# make A as origin
 
    B.x -= A.x
    B.y -= A.y
    P.x -= A.x
    P.y -= A.y

    # Determining cross Product
    """"The Cross-Product has an interesting Property which will be used to determine direction of a point from a line segment. That is, the cross-product of two points is positive if and only if the angle of those point at origin (0, 0) is in counter-clockwise. And conversely the cross-product is negative if and only if the angle of those point at origin is in clockwise direction."""
    cross_product = B.x * P.y - B.y * P.x
    # Return RIGHT if cross product is positive
    if (cross_product > 0):
        return RIGHT
        
    # Return LEFT if cross product is negative
    if (cross_product < 0):
        return LEFT

    # Return ZERO if cross product is zero
    return ZERO

def testLeftRightofLine(start,end,poi):
    d=(poi.x-start.x)*(end.y-start.y)-(poi.y-start.y)*(end.x-start.x)
    if d>0:
        #left
        return -1
    if d<0:
        #right
        return 1
    else:
        return 0

def findMotherRoad2AssociatedRoadDirection(mother_road,associated_road):
    left_buffered_line=mother_road.buffer(0.0001,single_sided=True)
#     right_buffered_line=direction_updated_roi.buffer(0.0007,single_sided=True)
    if left_buffered_line.intersects(associated_road.buffer(0.00001)) == True:
        direction=-1
#         print("left")
    else:
        direction=1
#         print("right")
    return direction

def addDirection_BuildingCode(direction,building_length,road_length,index,road_index):
    #right-even, left-odd number 
    # print(length)
    building_length=int(building_length)
    if road_length < 60:
        associated_road=road.iloc[road_index]

        if associated_road.mother_road == 'start':
            mother_road_code=associated_road.start
        else :
            mother_road_code=associated_road.end

        mother_road= road.loc[road['NEW'] == mother_road_code]

        mr_geom= mother_road.geometry
        d = {'col1': ['name1'], 'geometry': [Point(associated_road.start_point_x, associated_road.start_point_y)]}
        motherroad_start_point = {'col1': ['name1'], 'geometry': [Point(mother_road.start_point_x, mother_road.start_point_y)]}
        
        gdf = gpd.GeoDataFrame(d, crs=4326)
        mother_df = gpd.GeoDataFrame(motherroad_start_point, crs=4326)
        
        sp_point=gdf.iloc[0]
        mother_st_p=mother_df.iloc[0]
        for line in mr_geom:
            # print(line)
            for l in line:
                mother_line=l
                break

        result=split(mother_line,sp_point.geometry)
        for r  in result:
            splitted_line=r
            splitted_line.srid = 4326
            if splitted_line.contains(mother_st_p.geometry):
                break
            else:
                continue
        my_transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32644', always_xy=True)
        geom_transformed = ops.transform(my_transformer.transform, splitted_line)
        mother_road_direction=findMotherRoad2AssociatedRoadDirection(mother_road.geometry,associated_road.geometry)
        if (mother_road_direction == 1):
            # print("Right Direction")
            buildings.at[index,'from_mother_road_direction']="right"
            buildings.at[index,'road_length_d_code']=str(int(round_up_to_even(geom_transformed.length)))+"/"
        if (mother_road_direction == -1):
            # print("left Direction")
            buildings.at[index,'from_mother_road_direction']="left"
            buildings.at[index,'road_length_d_code']=str(int(round_up_to_odd(geom_transformed.length)))+"/"
            
        if (direction == 1):
            # print("Right Direction")
            buildings.at[index,'road_direction']="right"
            buildings.at[index,'road_length_d_code']+=str(int(round_up_to_even(building_length)))

        elif (direction == -1):
            # print("Left Direction")
            buildings.at[index,'road_direction']="left"
            buildings.at[index,'road_length_d_code']+=str(int(round_up_to_odd(building_length)))
        else:
            print("On  Line")
            buildings.at[index,'road_direction']="on_line"
    else :
        if (direction == 1):
            # print("Right Direction")
            buildings.at[index,'road_direction']="right"
            buildings.at[index,'road_length_d_code']=str(int(round_up_to_even(building_length)))

        elif (direction == -1):
            # print("Left Direction")
            buildings.at[index,'road_direction']="left"
            buildings.at[index,'road_length_d_code']=str(int(round_up_to_odd(building_length)))
        else:
            print("On  Line")
            buildings.at[index,'road_direction']="on_line"
    
    
#     print(buildings.at[index,'road_length_d_code'])
            
def getStartEndPoint(line,type):
    if len(line)==1 or type=="road" :
       
        df_line = line.geometry.boundary
        # print(len(df_line))
        # print(line_bound)
        # df_line=line_bound.explode(index_parts=True)
        start_point,end_point=df_line[0],df_line[1]

        return start_point,end_point
    else:
        raise ValueError ("Line contains more than one code")

def getLineLength(line):
    projected_line = line.to_crs(epsg=32644)
    length= projected_line.geometry.length
    float_length=length.tolist()
    return float_length

def round_up_to_even(f):
    return math.ceil(f / 2.) * 2

def round_up_to_odd(f):
    return math.ceil(f) // 2 * 2 + 1

def addStartEndPoint(start_point,end_point,index):
    
    road.at[index,'start_point_x']=float(start_point.x)
    road.at[index,'start_point_y']=float(start_point.y)
    
    road.at[index,'end_point_x']=float(end_point.x)
    road.at[index,'end_point_y']=float(end_point.y)

def ReviseStartEndDateWithBearing(bearing,index):
    # default direction is west to east , south to north 
    # print("ma revise garna aako dubai vettera")
    if bearing is None :
        bearing=0
    bearing=int(bearing)
    def assign_start():
        print("start")
        
        road.at[index,'mother_road']='start'

    def assign_end():
    # east_west/west_east direction , assign to east
        print("end")
        road.at[index,'mother_road']='end'

    
    if ((bearing >= 0) and (bearing<=45)) :
        #    North_South/south_north Direction, assign to south
        
        assign_start()
        
    if ((bearing > 45) and (bearing<135)) :
        
        # east_west/west_east direction , assign to east
        assign_end()
    
    if ((bearing >= 135) and (bearing<=225)) :
        #    North_South/south_north Direction, assign to south
        
        assign_start()
    if ((bearing > 225) and (bearing<315)) :
            
        # east_west/west_east direction , assign to east
        assign_end()
    if ((bearing >= 315) and (bearing<=360)) :
    #    North_South/south_north Direction, assign to south
        
        assign_start()      

def FindIntersectedFeatures(start_point,end_point,line_row,index_road):
    for index, row in road.iterrows():
        line=row.geometry        
        if str(row.NEW) != str(line_row.NEW):
            if line.distance(start_point) < 1e-8 :
                # print("start found "+ str(row.NEW))
                # print("mero start vetiyo guys ")
                road.at[index_road,'start']=str(row.NEW)
    
            if line.distance(end_point) < 1e-8:
                # print("end found "+ str(row.NEW))
                road.at[index_road,'end']=str(row.NEW)
                # print("mero end vetiyo guys ")


def scan_I_feature(line_row,index_road):
    
        if ((line_row.start!= None) and (line_row.end== None)) :
            road.at[index_road,'mother_road']='start'
        if ((line_row.start== None) and (line_row.end != None)) :
            road.at[index_road,'mother_road']='end'
    
        if ((line_row.start!= None) and (line_row.end != None)) :
        #both start and end exist which means line is in between roads 
            road.at[index_road,'mother_road']='None'
            ReviseStartEndDateWithBearing(line_row.bearing,index_road)

                    
def CalculateBearing(start_point,end_point,index):
    brng = Geodesic.WGS84.Inverse(start_point.y, start_point.x, end_point.y, end_point.x)['azi1']
    road.at[index,'bearing']=brng

def is_number(string):
    try:
        float(string)
        return True
    except ValueError:
        return False    
    
def AssignIntersectedPoint(start_point_assumed,end_point_assumed,row,index):
#     print(row)
    if  row.mother_road :
        mother_road_direction = row.mother_road
        mother_road_code=None
        if mother_road_direction == 'start':
            mother_road_code=row.start
        elif mother_road_direction == 'end' :
            mother_road_code=row.end
        mother_road= road.loc[road['NEW'] == mother_road_code].geometry
        current_road = row.geometry
        mother_road_geom=mother_road
        for l in mother_road:
            for single in l:
                mother_road_geom=single
            break
#         print(mother_road_geom)
        for l in current_road:
            road_geom=l
            break
#         print(road_geom)
        start_point=mother_road_geom.intersection(road_geom)
        print(start_point)
        st=str(type(start_point))
#         print(st)
#         geoseries=f"""<class 'geopandas.geoseries.GeoSeries'>"""
#         print(geoseries)
    
        if isinstance(start_point,pandas.core.series.Series) :
            print(type(start_point))
            print('not counted')
            
            road.at[index,'intersected']='No'
        else:
            print(start_point)
            try:
                x=start_point.x
                y=start_point.y
                road.at[index,'start_point_x']=float(x)
                road.at[index,'start_point_y']=float(y)              
            except:
                try:
                    for p in start_point:
                        x=p.x
                        y=p.y
                        break
                    road.at[index,'start_point_x']=x
                    road.at[index,'start_point_y']=y
                except:
                    try:
                        df_line = start_point.boundary
                        start=df_line[0]
                        road.at[index,'start_point_x']=start.x
                        road.at[index,'start_point_y']=start.y
                    except:
                        print('i didnt assigned')
                        pass
#                     road.at[index,'start_point_x']=0
#                     road.at[index,'start_point_y']=0
 

        
def get_building_projected_road_length(road_linked_wrt_point,row):
    road_code = row.NEW
#     print(road_linked_wrt_point)
    linked_road_geometry = road_linked_wrt_point.geometry
    building_geometry = row.geometry
    degree_length_for_building=linked_road_geometry.project(building_geometry)
    # 5m=0.00005 in degree 
    building_point_projected_on_road=linked_road_geometry.interpolate(degree_length_for_building) #end_always 
    for r in building_point_projected_on_road:
        building_point_projected_on_road_single_geom=r
        break
    try:
        projected_building_on_road = {'col1': ['name1'], 'geometry': [Point(building_point_projected_on_road_single_geom.x, building_point_projected_on_road_single_geom.y)]}
    except:
        print(projected_building_on_road)
        raise("found error")

    road_link_wrt_point_x=road_linked_wrt_point.start_point_x
    road_link_wrt_point_y=road_linked_wrt_point.start_point_y
    try:
        road_start_point = {'col1': ['name1'], 'geometry': [Point(road_link_wrt_point_x,road_link_wrt_point_y)]}

        projected_building_on_road_gdf = gpd.GeoDataFrame(projected_building_on_road, crs=4326)


        road_start_df = gpd.GeoDataFrame(road_start_point, crs=4326)


        splitting_point=projected_building_on_road_gdf.iloc[0]
        start_point=road_start_df.iloc[0]
#         print(start_point.geometry)
        for line in linked_road_geometry:
            # print(line)
            for l in line:
                mother_line=l
                break
        result=split(mother_line,splitting_point.geometry.buffer(0.0000001))

        for r  in result:
                splitted_line=r
                splitted_line.srid = 4326
                if splitted_line.buffer(0.00001).contains(start_point.geometry.buffer(0.0000001)) == True:

                    break
                else:
                    continue


        
        my_transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32644', always_xy=True)
        geom_transformed = ops.transform(my_transformer.transform, splitted_line)

        clipped_length=geom_transformed.length
        return clipped_length ,splitted_line ,building_point_projected_on_road_single_geom,start_point.geometry,degree_length_for_building
    except:
        pass



    
                   


In [5]:
def CalculateTouchLine(index,row):
    
    # print(row)
    start_point,end_point=getStartEndPoint(row,"road")
    addStartEndPoint(start_point,end_point,index)
    CalculateBearing(start_point,end_point,index)
    FindIntersectedFeatures(start_point,end_point,row,index)
    scan_I_feature(row,index)
    AssignIntersectedPoint(start_point,end_point,row,index)
    # break


In [8]:
for index, row in road.iterrows ():
    CalculateTouchLine(index,row)
    print(f"""i: {index}""")
    

GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
i: 0
end
POINT (83.47182876258739 27.59323857200252)
POINT (83.47182876258739 27.59323857200252)
i: 1
POINT (83.47268607010614 27.60246986818993)
POINT (83.47268607010614 27.60246986818993)
i: 2
POINT (83.47040628242405 27.57787412609008)
POINT (83.47040628242405 27.57787412609008)
i: 3
POINT (83.47172783257349 27.59225408505802)
POINT (83.47172783257349 27.59225408505802)
i: 4
end
POINT (83.46736454413843 27.56698659255727)
POINT (83.46736454413843 27.56698659255727)
i: 5
POINT (83.47153400529015 27.59036561757927)
POINT (83.47153400529015 27.59036561757927)
i: 6
end
POINT (83.47187124206425 27.59364565192533)
POINT (83.47187124206425 27.59364565192533)
i: 7
end
POINT (83.47263962641767 27.60196558594311)
POINT (83.47263962641767 27.60196558594311)
i: 8
POINT (83.47213499163661 27.59644264511201)
POINT (83.47213499163661 27.59644264511201)
i: 9
POINT (83.47071132616685 27.58135272534804)
POINT (83.47071

POINT (83.46409408127687 27.60264427461362)
POINT (83.46409408127687 27.60264427461362)
i: 85
end
MULTIPOINT (83.46945272316469 27.58678523215281, 83.47021403794753 27.5861709277462)
MULTIPOINT (83.46945272316469 27.58678523215281, 83.47021403794753 27.5861709277462)
i: 86
start
POINT (83.46930570919119 27.5946145949876)
POINT (83.46930570919119 27.5946145949876)
i: 87
POINT (83.46806967652928 27.57928846131166)
POINT (83.46806967652928 27.57928846131166)
i: 88
start
POINT (83.45892516306589 27.56683897154136)
POINT (83.45892516306589 27.56683897154136)
i: 89
POINT (83.47150689522709 27.59260663009195)
POINT (83.47150689522709 27.59260663009195)
i: 90
POINT (83.4702157331696 27.59357258021043)
POINT (83.4702157331696 27.59357258021043)
i: 91
POINT (83.46215994622457 27.57253243552356)
POINT (83.46215994622457 27.57253243552356)
i: 92
start
POINT (83.46231292090465 27.56762419390145)
POINT (83.46231292090465 27.56762419390145)
i: 93
GeoSeries([], dtype: geometry)
<class 'geopandas.geose

start
POINT (83.46079436976441 27.58428014900971)
POINT (83.46079436976441 27.58428014900971)
i: 169
POINT (83.43969330140482 27.59592745678674)
POINT (83.43969330140482 27.59592745678674)
i: 170
POINT (83.45864736968042 27.5734361417646)
POINT (83.45864736968042 27.5734361417646)
i: 171
POINT (83.43984491810852 27.59583754436721)
POINT (83.43984491810852 27.59583754436721)
i: 172
POINT (83.45459247316415 27.59034999995259)
POINT (83.45459247316415 27.59034999995259)
i: 173
end
POINT (83.46031340064297 27.5741794943882)
POINT (83.46031340064297 27.5741794943882)
i: 174
POINT (83.45914828396621 27.57330037741053)
POINT (83.45914828396621 27.57330037741053)
i: 175
POINT (83.45997992213574 27.58453232250872)
POINT (83.45997992213574 27.58453232250872)
i: 176
start
POINT (83.45543963812884 27.5938612392049)
POINT (83.45543963812884 27.5938612392049)
i: 177
start
POINT (83.43780754538199 27.59591046139872)
POINT (83.43780754538199 27.59591046139872)
i: 178
end
POINT (83.45892516306589 27.56

POINT (83.46270805783064 27.57742490393622)
POINT (83.46270805783064 27.57742490393622)
i: 255
start
POINT (83.46384818864306 27.58277196525404)
POINT (83.46384818864306 27.58277196525404)
i: 256
POINT (83.46717947625189 27.60665606482559)
POINT (83.46717947625189 27.60665606482559)
i: 257
end
POINT (83.46905658439391 27.60298940383797)
POINT (83.46905658439391 27.60298940383797)
i: 258
start
POINT (83.46759247821035 27.59867287745141)
POINT (83.46759247821035 27.59867287745141)
i: 259
end
POINT (83.46304003536937 27.5820388091438)
POINT (83.46304003536937 27.5820388091438)
i: 260
POINT (83.46439721306098 27.5898755032519)
POINT (83.46439721306098 27.5898755032519)
i: 261
POINT (83.46987896674096 27.58024431902015)
POINT (83.46987896674096 27.58024431902015)
i: 262
POINT (83.46165059270078 27.57643070251498)
POINT (83.46165059270078 27.57643070251498)
i: 263
POINT (83.46424754698677 27.57094090799654)
POINT (83.46424754698677 27.57094090799654)
i: 264
end
POINT (83.46641435104152 27.59

POINT (83.466986702974 27.58818580303245)
POINT (83.466986702974 27.58818580303245)
i: 339
POINT (83.46182581840685 27.57691377335203)
POINT (83.46182581840685 27.57691377335203)
i: 340
end
MULTIPOINT (83.46577314701318 27.56919417726954, 83.46717383480467 27.56974050462065)
MULTIPOINT (83.46577314701318 27.56919417726954, 83.46717383480467 27.56974050462065)
i: 341
POINT (83.46641435104152 27.59787895774775)
POINT (83.46641435104152 27.59787895774775)
i: 342
POINT (83.46549540668775 27.5693259864052)
POINT (83.46549540668775 27.5693259864052)
i: 343
end
POINT (83.4690080479831 27.59301898903516)
POINT (83.4690080479831 27.59301898903516)
i: 344
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
i: 345
POINT (83.46458372795672 27.57234261022336)
POINT (83.46458372795672 27.57234261022336)
i: 346
end
POINT (83.46898583113125 27.6002747399873)
POINT (83.46898583113125 27.6002747399873)
i: 347
POINT (83.47086785137031 27.60149673428691)
POINT (83.4708678513

...

i: 505
POINT (83.46244568961691 27.57812763148507)
POINT (83.46244568961691 27.57812763148507)
i: 506
POINT (83.46502471052202 27.58980113561398)
POINT (83.46502471052202 27.58980113561398)
i: 507
POINT (83.46830683938919 27.59550533570274)
POINT (83.46830683938919 27.59550533570274)
i: 508
start
POINT (83.46523515547813 27.58974606832646)
POINT (83.46523515547813 27.58974606832646)
i: 509
POINT (83.46798387035938 27.58000348867012)
POINT (83.46798387035938 27.58000348867012)
i: 510
POINT (83.46433043570113 27.58958547459002)
POINT (83.46433043570113 27.58958547459002)
i: 511
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
i: 512
POINT (83.4675750817247 27.57502455493932)
POINT (83.4675750817247 27.57502455493932)
i: 513
POINT (83.46474651154233 27.56877825161507)
POINT (83.46474651154233 27.56877825161507)
i: 514
LINESTRING (83.46470360039092 27.589787614307, 83.46471373125377 27.58983193199811)
LINESTRING (83.46470360039092 27.589787614307, 83.46471

POINT (83.45953458595244 27.57441454659408)
POINT (83.45953458595244 27.57441454659408)
i: 593
POINT (83.46130014128858 27.6055477529265)
POINT (83.46130014128858 27.6055477529265)
i: 594
end
POINT (83.46213164366048 27.6061494920055)
POINT (83.46213164366048 27.6061494920055)
i: 595
POINT (83.45487302027374 27.60659482279289)
POINT (83.45487302027374 27.60659482279289)
i: 596
end
POINT (83.44047797787641 27.59921796014078)
POINT (83.44047797787641 27.59921796014078)
i: 597
POINT (83.43294214656288 27.59816495474894)
POINT (83.43294214656288 27.59816495474894)
i: 598
POINT (83.45853827206005 27.60207259362004)
POINT (83.45853827206005 27.60207259362004)
i: 599
POINT (83.43421557146964 27.60388551280935)
POINT (83.43421557146964 27.60388551280935)
i: 600
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
i: 601
start
MULTIPOINT (83.44379644205219 27.56375779528605, 83.44680175329836 27.56804318264682)
MULTIPOINT (83.44379644205219 27.56375779528605, 83.44

i: 680
POINT (83.45044484485379 27.57920657872069)
POINT (83.45044484485379 27.57920657872069)
i: 681
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
i: 682
end
MULTIPOINT (83.42559174915993 27.59265597319467, 83.42625869088596 27.59296979252196)
MULTIPOINT (83.42559174915993 27.59265597319467, 83.42625869088596 27.59296979252196)
i: 683
end
POINT (83.4527008616555 27.57518962190358)
POINT (83.4527008616555 27.57518962190358)
i: 684
start
POINT (83.43297050308632 27.58761966541527)
POINT (83.43297050308632 27.58761966541527)
i: 685
end
LINESTRING EMPTY
LINESTRING EMPTY
i didnt assigned
i: 686
start
POINT (83.44175083873029 27.56616402674848)
POINT (83.44175083873029 27.56616402674848)
i: 687
end
POINT (83.43252731178609 27.57872557452572)
POINT (83.43252731178609 27.57872557452572)
i: 688
POINT (83.43934977836966 27.59061889004977)
POINT (83.43934977836966 27.59061889004977)
i: 689
POINT (83.4391962335194 27.58844983409261)
POINT (83.4391962335194 27.

POINT (83.42102282174591 27.56561971297941)
i: 764
POINT (83.44483490900393 27.58470312175172)
POINT (83.44483490900393 27.58470312175172)
i: 765
POINT (83.42879434108602 27.58610148839301)
POINT (83.42879434108602 27.58610148839301)
i: 766
POINT (83.42598189484863 27.58861521851759)
POINT (83.42598189484863 27.58861521851759)
i: 767
end
POINT (83.43591666502215 27.56715490787275)
POINT (83.43591666502215 27.56715490787275)
i: 768
POINT (83.44348229537167 27.5725463579281)
POINT (83.44348229537167 27.5725463579281)
i: 769
end
POINT (83.43684983844935 27.56790938061346)
POINT (83.43684983844935 27.56790938061346)
i: 770
POINT (83.42595788834592 27.57124368544777)
POINT (83.42595788834592 27.57124368544777)
i: 771
POINT (83.4314391159225 27.57662966672581)
POINT (83.4314391159225 27.57662966672581)
i: 772
POINT (83.44050570487445 27.57487129337557)
POINT (83.44050570487445 27.57487129337557)
i: 773
POINT (83.4216565515087 27.57880549367866)
POINT (83.4216565515087 27.57880549367866)
i: 7

POINT (83.44967369058838 27.60685002610785)
POINT (83.44967369058838 27.60685002610785)
i: 853
end
POINT (83.45151996367542 27.60629622089391)
POINT (83.45151996367542 27.60629622089391)
i: 854
POINT (83.44973302425973 27.6067419878525)
POINT (83.44973302425973 27.6067419878525)
i: 855
POINT (83.44964634395987 27.6049144737344)
POINT (83.44964634395987 27.6049144737344)
i: 856
POINT (83.45059089385472 27.60667210063701)
POINT (83.45059089385472 27.60667210063701)
i: 857
POINT (83.45102996805866 27.60638500916087)
POINT (83.45102996805866 27.60638500916087)
i: 858
start
POINT (83.45039317520491 27.60752150401424)
POINT (83.45039317520491 27.60752150401424)
i: 859


In [7]:
row_fetched=0
while row_fetched<len(road.index):
    start=row_fetched
    end = row_fetched+10
    for i in range(start,end):
        row=road.iloc[i]
        try:
            CalculateTouchLine(i,row)
            print(f"""i: {i}""")
        except:
            continue
    row_fetched=end
    print(f"""Batch Done, {end}""")
print("Sucessfully added : startpoint, endpoint , Start touching Line , End Touching Line , bearing ")   

GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
Batch Done, 10
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geo

GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
start
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
Batch Done, 100
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoserie

GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
start
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
start
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
Batch Done, 200
start
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
start
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
start
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
start
GeoSeries([], dtype: geometry)


GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
Batch Done, 300
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.

start
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
start
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
Batch Done, 400
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
start
GeoSeries([], dtype: geometry)
<class 'geopan

start
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
Batch Done, 490
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
start
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
start
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class

GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
Batch Done, 590
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
start
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoserie

GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
start
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
start
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
Batch Done, 690
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
start
GeoSeries([], dtype: geometry)
<class

end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
Batch Done, 780
end
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
start
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
GeoSeries([], dtype: geometry)
<class 'geopandas.geoseries.GeoSeries'>
not counted
start
GeoSeries([], dtype: geometry)
<class 'geopandas.ge

In [9]:
def assign_building_code(index,row):  

    road_linked_wrt_point= road.loc[road['NEW'] == row.NEW]
#     print(road_linked_wrt_point)
    for l in road_linked_wrt_point.geometry:
        line_geom=l
        break

    my_transformer = Transformer.from_crs('EPSG:4326', 'EPSG:32644', always_xy=True)
    geom_transformed = ops.transform(my_transformer.transform, line_geom)

    road_length=geom_transformed.length

    clipped_length,splitted_mother_road_side,building_point_projected_on_road_single_geom,start_point,degree_length_for_building=get_building_projected_road_length(road_linked_wrt_point,row)

    splitted_with_buffer_boi=split(splitted_mother_road_side,building_point_projected_on_road_single_geom.buffer(0.0001)) 

    for r  in splitted_with_buffer_boi:
        roi_line=r
        roi_line.srid = 4326
        if roi_line.buffer(0.00001).contains(start_point.buffer(0.0000001)) == False:
            break
        else:
            continue
#     print(roi_line)
#     print(shapely.geometry.mapping(roi_line))
    df_line = roi_line.boundary
    start_roi,end_roi=df_line[0],df_line[1]
#     print(start_roi)
#     print(end_roi)
#     print(building_point_projected_on_road_single_geom)
    if (round(start_roi.x,6) == round(building_point_projected_on_road_single_geom.x,4)) and (round(start_roi.y,4) == round(building_point_projected_on_road_single_geom.y,4)) :
        start_roi=end_roi
        end_roi=building_point_projected_on_road_single_geom
        print("start_end changed")

    road_index=road_linked_wrt_point.index[0]
#         road.at[road_index,'length']=length[0]       

    poi = row.geometry
    #constructing new geometry with new start and end for checking left and right 
    direction_updated_roi=LineString([start_roi,end_roi])

    left_buffered_line=direction_updated_roi.buffer(0.01,single_sided=True)
#     right_buffered_line=direction_updated_roi.buffer(0.0007,single_sided=True)
    if left_buffered_line.intersects(poi.buffer(0.00004)) == True:
        direction=-1
#         print("left")
    else:
        direction=1
#         print("right")
        
#     print(shapely.geometry.mapping(left_buffered_line))
#     print(shapely.geometry.mapping(poi.buffer(0.00004)))
    
    # break
#     print(clipped_length)
#     print(direction)
    buildings.at[index,'distance2_intersecting_road']=clipped_length
    road.at[road_index,'length']=road_length
    
    
    addDirection_BuildingCode(direction,clipped_length,road_length,index,road_index)
    
        
#     print("added building code")


In [ ]:
i=2576
row=buildings.iloc[i]
assign_building_code(i,row)
print(buildings.iloc[i])

In [11]:
row_fetched=0
while row_fetched<len(buildings.index):
    start=row_fetched
    end = row_fetched+100
    
    for i in range(start,end):
        row=buildings.iloc[i]
        try:
            assign_building_code(i,row)
            print(buildings.iloc[i].distance2_intersecting_road)
            print((f"""i: {i}"""))
        except:
            continue
    print(f"""Batch Done, {end}""")
    row_fetched=end

C:\Users\Kshitiz\AppData\Local\Temp/ipykernel_9464/2296941967.py:330: UserWarning: Geometry is in a geographic CRS. Results from 'interpolate' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  building_point_projected_on_road=linked_road_geometry.interpolate(degree_length_for_building) #end_always


669
i: 0
659
i: 1
650
i: 2
206
i: 3
214
i: 4
214
i: 5
193
i: 6
185
i: 7
181
i: 8
214
i: 9
214
i: 10


C:\Users\Kshitiz\AppData\Local\Temp/ipykernel_9464/2296941967.py:61: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  left_buffered_line=mother_road.buffer(0.0001,single_sided=True)


169
i: 14
170
i: 15
173
i: 16
166
i: 17
178
i: 18
188
i: 19
197
i: 20
202
i: 21
215
i: 22
213
i: 23
226
i: 24
232
i: 25
243
i: 26
243
i: 27
252
i: 28
262
i: 29
271
i: 30
277
i: 31
260
i: 32
290
i: 33
288
i: 34
304
i: 35
313
i: 36
322
i: 37
948
i: 38
14
i: 39
368
i: 40
383
i: 41
359
i: 42
922
i: 43
916
i: 44
910
i: 45
404
i: 46
279
i: 47
424
i: 48
175
i: 49
179
i: 50
424
i: 51
449
i: 52
453
i: 53
460
i: 54
21
i: 55
17
i: 56
31
i: 57
49
i: 58
65
i: 59
92
i: 60
107
i: 61
157
i: 62
105
i: 63
38
i: 64
210
i: 65
191
i: 66
185
i: 67
177
i: 68
171
i: 69
218
i: 70
210
i: 71
199
i: 72
187
i: 73
178
i: 74
562
i: 75
63
i: 76
63
i: 77
60
i: 78
613
i: 79
618
i: 80
635
i: 81
630
i: 82
618
i: 83
141
i: 85
128
i: 86
116
i: 87
108
i: 88
96
i: 89
82
i: 90
117
i: 91
135
i: 93
59
i: 94
63
i: 95
52
i: 96
40
i: 97
32
i: 98
55
i: 99
Batch Done, 100
71
i: 100
100
i: 101
21
i: 102
15
i: 103
1
i: 104
1140
i: 105
1151
i: 106
1164
i: 107
181
i: 109
1062
i: 112
1152
i: 113
1179
i: 114
1066
i: 115
13
i: 116
11
i: 11

353
i: 900
369
i: 901
377
i: 902
208
i: 903
266
i: 904
276
i: 905
292
i: 906
330
i: 907
359
i: 908
359
i: 909
377
i: 910
52
i: 911
509
i: 912
648
i: 913
648
i: 914
509
i: 915
572
i: 916
556
i: 917
531
i: 918
528
i: 919
516
i: 920
491
i: 921
471
i: 922
469
i: 923
543
i: 924
558
i: 925
469
i: 926
444
i: 927
441
i: 928
422
i: 929
412
i: 930
422
i: 931
391
i: 932
388
i: 933
368
i: 934
370
i: 935
379
i: 936
395
i: 937
374
i: 938
352
i: 939
324
i: 940
338
i: 941
305
i: 942
286
i: 943
257
i: 944
228
i: 945
271
i: 946
187
i: 947
113
i: 950
86
i: 951
49
i: 952
68
i: 953
13
i: 954
1091
i: 955
1101
i: 956
522
i: 957
513
i: 958
506
i: 959
499
i: 960
1078
i: 961
1070
i: 962
1061
i: 963
549
i: 964
557
i: 965
569
i: 966
578
i: 967
579
i: 968
585
i: 969
590
i: 970
612
i: 971
624
i: 972
629
i: 973
638
i: 974
630
i: 975
650
i: 976
654
i: 977
657
i: 978
666
i: 979
642
i: 980
641
i: 981
622
i: 982
666
i: 983
672
i: 984
682
i: 985
684
i: 986
690
i: 987
704
i: 988
715
i: 989
701
i: 990
708
i: 991
720
i: 992

...

219
i: 3910
144
i: 3911
3525
i: 3912
8
i: 3913
28
i: 3914
73
i: 3915
62
i: 3916
86
i: 3917
3176
i: 3918
3163
i: 3919
3152
i: 3920
3116
i: 3921
3108
i: 3922
3084
i: 3923
10
i: 3924
51
i: 3925
27
i: 3926
33
i: 3927
3064
i: 3928
128
i: 3929
107
i: 3930
104
i: 3931
94
i: 3932
114
i: 3933
981
i: 3934
961
i: 3935
949
i: 3936
915
i: 3937
940
i: 3938
877
i: 3939
29
i: 3940
12
i: 3941
859
i: 3942
833
i: 3943
838
i: 3944
857
i: 3945
200
i: 3946
155
i: 3947
800
i: 3948
698
i: 3949
223
i: 3950
219
i: 3951
139
i: 3952
57
i: 3953
72
i: 3954
73
i: 3955
54
i: 3956
81
i: 3957
70
i: 3958
32
i: 3959
26
i: 3960
1654
i: 3961
1641
i: 3962
1631
i: 3963
70
i: 3964
65
i: 3965
40
i: 3966
33
i: 3967
1598
i: 3968
1726
i: 3969
1723
i: 3970
1676
i: 3971
1669
i: 3972
1622
i: 3973
1614
i: 3974
1601
i: 3975
291
i: 3976
292
i: 3977
273
i: 3978
261
i: 3979
1561
i: 3980
273
i: 3981
248
i: 3982
237
i: 3983
230
i: 3984
32
i: 3985
13
i: 3986
31
i: 3987
65
i: 3988
59
i: 3989
1470
i: 3990
1466
i: 3991
70
i: 3992
1447
i: 3993


1380
i: 4582
1372
i: 4583
1350
i: 4584
1343
i: 4585
1338
i: 4586
1353
i: 4587
1360
i: 4588
1361
i: 4589
1363
i: 4590
1367
i: 4591
413
i: 4592
13
i: 4593
13
i: 4594
35
i: 4595
440
i: 4596
298
i: 4597
297
i: 4598
297
i: 4599
Batch Done, 4600
306
i: 4600
515
i: 4601
508
i: 4602
537
i: 4603
553
i: 4604
551
i: 4605
563
i: 4606
566
i: 4607
574
i: 4608
558
i: 4609
590
i: 4610
601
i: 4611
599
i: 4612
564
i: 4613
389
i: 4614
374
i: 4615
365
i: 4616
357
i: 4617
337
i: 4618
315
i: 4619
290
i: 4620
151
i: 4621
177
i: 4622
182
i: 4623
189
i: 4624
217
i: 4625
214
i: 4626
109
i: 4627
186
i: 4628
218
i: 4629
241
i: 4630
230
i: 4631
220
i: 4632
226
i: 4633
262
i: 4634
246
i: 4635
268
i: 4636
272
i: 4637
270
i: 4638
330
i: 4639
117
i: 4640
57
i: 4641
51
i: 4642
34
i: 4643
22
i: 4644
877
i: 4645
183
i: 4646
166
i: 4647
1019
i: 4648
1033
i: 4649
224
i: 4650
209
i: 4651
235
i: 4652
238
i: 4653
254
i: 4654
250
i: 4655
262
i: 4656
274
i: 4657
244
i: 4658
134
i: 4659
start_end changed
115
i: 4660
99
i: 4661
1

568
i: 5287
781
i: 5288
771
i: 5289
774
i: 5290
766
i: 5291
759
i: 5292
745
i: 5293
778
i: 5294
521
i: 5295
508
i: 5296
495
i: 5297
496
i: 5298
506
i: 5299
Batch Done, 5300
497
i: 5300
499
i: 5301
484
i: 5302
469
i: 5303
450
i: 5304
461
i: 5305
465
i: 5306
469
i: 5307
425
i: 5308
451
i: 5309
397
i: 5310
385
i: 5311
379
i: 5312
365
i: 5313
350
i: 5314
340
i: 5315
328
i: 5316
325
i: 5317
137
i: 5318
120
i: 5319
101
i: 5320
78
i: 5321
145
i: 5327
190
i: 5328
86
i: 5329
33
i: 5330
21
i: 5331
53
i: 5332
440
i: 5333
457
i: 5334
500
i: 5335
52
i: 5336
555
i: 5337
19
i: 5338
107
i: 5339
80
i: 5340
590
i: 5341
713
i: 5342
703
i: 5343
113
i: 5344
658
i: 5345
41
i: 5346
43
i: 5347
143
i: 5348
119
i: 5349
528
i: 5350
536
i: 5351
505
i: 5352
395
i: 5353
448
i: 5354
366
i: 5355
448
i: 5356
28
i: 5357
29
i: 5358
21
i: 5359
11
i: 5360
3
i: 5361
14
i: 5362
108
i: 5363
116
i: 5364
23
i: 5365
28
i: 5366
35
i: 5367
43
i: 5368
40
i: 5369
50
i: 5370
43
i: 5371
62
i: 5372
72
i: 5373
53
i: 5374
45
i: 5375
38


347
i: 5985
339
i: 5986
346
i: 5987
334
i: 5988
338
i: 5989
3867
i: 5990
3857
i: 5991
326
i: 5992
319
i: 5993
312
i: 5994
304
i: 5996
308
i: 5997
3840
i: 5998
Batch Done, 6000
66
i: 6000
60
i: 6001
3811
i: 6002
3805
i: 6003
3801
i: 6004
3796
i: 6005
12
i: 6006
38
i: 6007
8
i: 6008
21
i: 6009
28
i: 6010
183
i: 6013
156
i: 6014
131
i: 6015
121
i: 6016
116
i: 6017
107
i: 6018
62
i: 6019
67
i: 6020
60
i: 6021
17
i: 6022
3735
i: 6023
3728
i: 6024
3707
i: 6025
18
i: 6026
33
i: 6027
32
i: 6028
24
i: 6029
48
i: 6036
63
i: 6037
52
i: 6038
3776
i: 6039
44
i: 6040
3670
i: 6041
57
i: 6042
1
i: 6043
10
i: 6044
34
i: 6045
33
i: 6046
45
i: 6047
68
i: 6048
80
i: 6049
72
i: 6050
10
i: 6051
69
i: 6052
57
i: 6053
83
i: 6054
119
i: 6056
3563
i: 6057
3539
i: 6058
3515
i: 6059
3458
i: 6060
14
i: 6061
37
i: 6062
13
i: 6063
33
i: 6064
53
i: 6065
58
i: 6066
76
i: 6067
83
i: 6068
88
i: 6069
80
i: 6070
71
i: 6071
65
i: 6072
64
i: 6073
70
i: 6074
59
i: 6075
48
i: 6076
90
i: 6077
87
i: 6078
74
i: 6079
91
i: 6080
1

IndexError: single positional indexer is out-of-bounds

In [ ]:
buildings.to_file("output_building_1.geojson", driver="GeoJSON")

In [ ]:
print(buildings)

In [ ]:
road.to_file("output_road_1.geojson", driver="GeoJSON")